### Вводные
В БД имеется две таблицы:
 * feed_actions
 * message_actions<br>
<b>feed_actions</b> несет в себе информацию о том, как пользователь взаимодействал с постом (просмотрел и/или поставил лайк).
<b>message_actions</b> - информация по статистике встроенного мессенжера социальной сети, которая содержит информацию о количестве отправленных и полученных сообщениях.
#### Задача
Автоматизировать обработку данных следующим образом:
* в feed_actions для каждого пользователя считать число просмотров и лайков контента.
* В message_actions для каждого пользователя считать, сколько он получает и отсылает сообщений, скольким людям он пишет, сколько людей пишут ему.
* Объединить две таблицы в одну и считать все метрики в разрезе по полу, возрасту и ОС
* Финальные данные со всеми метриками записать в отдельную таблицу в ClickHouse.
* Каждый день таблица должна дополняться новыми данными. 

In [1]:
import configparser

from datetime import datetime, timedelta
import pandas as pd
import pandahouse as ph


In [2]:
# Подсоединяемся к нужной базе данных.
# Логины, пароли и прочее в отдельном файле
config = configparser.ConfigParser()
config.read('config.ini')

connection = {
    'host': config.get('clickhouse_connection', 'host'),
    'user': config.get('clickhouse_connection', 'user'),
    'password': config.get('clickhouse_connection', 'password'),
    'database': config.get('clickhouse_connection', 'database')
}

### Состав таблицы feed_actions

In [3]:
feed_actions = '''
            select *
            from simulator_20240320.feed_actions
            order by time
            limit 100
            
        '''
ph.read_clickhouse(feed_actions, connection=connection).head()

,user_id,post_id,action,time,gender,age,country,city,os,source,exp_group
0,106791,76,view,2024-02-03 00:26:19,0,45,Russia,Chelyabinsk,iOS,ads,3
1,659,76,view,2024-02-03 00:26:31,0,30,Russia,Partizansk,Android,ads,0
2,106791,76,like,2024-02-03 00:27:59,0,45,Russia,Chelyabinsk,iOS,ads,3
3,695,76,view,2024-02-03 00:29:39,1,27,Russia,Engels,Android,ads,0
4,695,76,like,2024-02-03 00:29:51,1,27,Russia,Engels,Android,ads,0


### Состав таблицы message_actions

In [5]:
message_actions  = '''
            select *
            from simulator_20240320.message_actions 
            limit 100
        '''
ph.read_clickhouse(message_actions, connection=connection).head()

,user_id,receiver_id,time,source,exp_group,gender,age,country,city,os
0,16117,21886,2024-05-13 00:00:01,ads,0,1,21,Russia,Tuymazy,Android
1,16117,24425,2024-05-13 00:00:01,ads,0,1,21,Russia,Tuymazy,Android
2,16117,110592,2024-05-13 00:00:01,ads,0,1,21,Russia,Tuymazy,Android
3,16117,119775,2024-05-13 00:00:01,ads,0,1,21,Russia,Tuymazy,Android
4,16117,136173,2024-05-13 00:00:01,ads,0,1,21,Russia,Tuymazy,Android


In [6]:
        # SQL запрос для данных из feed_actions
feed_query = '''
            select  toDate(time) as event_date,
                    user_id,
                    sum(action = 'like') as likes,
                    sum(action = 'view') as views,
                    max(gender) as gender,
                    max(age) as age,
                    max(os) as os
            from simulator_20240320.feed_actions
            where toDate(time)+10 = yesterday()
            group by user_id, event_date
        '''
feed_data = ph.read_clickhouse(feed_query, connection=connection)

In [8]:
  message_query = '''
            select  toDate(time) as event_date,
                    user_id,
                    count(receiver_id) as messages_sent,
                    countIf(user_id in  (select receiver_id from simulator_20240320.message_actions
                    where toDate(time)+10 = yesterday())
                    ) as messages_received,
                    count(distinct receiver_id) as users_sent,
                    countIf(user_id in  (select distinct receiver_id from simulator_20240320.message_actions
                    where toDate(time)+10 = yesterday())
                    ) as users_received,
                    max(gender) as gender,
                    max(age) as age,
                    max(os) as os
            from simulator_20240320.message_actions
            where toDate(time)+10 = yesterday() 
            group by user_id, event_date
        '''

In [9]:
message_data = ph.read_clickhouse(message_query, connection=connection)

In [10]:
combined_data = feed_data.merge(message_data, on=['event_date', 'user_id', 'gender', 'age', 'os'], how='outer')

In [11]:
 gender_stats = combined_data.groupby(['event_date', 'gender'])\
            .agg(views=('views', 'sum'),
                 likes=('likes', 'sum'),
                 messages_received=('messages_received', 'sum'),
                 messages_sent=('messages_sent', 'sum'),
                 users_received=('users_received', 'sum'),
                 users_sent=('users_sent', 'sum')
                )\
            .reset_index()\
            .rename(columns={'gender': 'dimension_value'})
       

In [12]:
gender_stats.dimension_value = gender_stats.dimension_value.astype(str)
gender_stats.insert(loc=1, column='dimension', value='gender')      

### Итоговый вывод пайплайна - таблица со срезами по данным

In [13]:
gender_stats

,event_date,dimension,dimension_value,views,likes,messages_received,messages_sent,users_received,users_sent
0,2024-06-29,gender,0,294545.0,59022.0,3726.0,5627.0,3726.0,4479.0
1,2024-06-29,gender,1,356064.0,71506.0,4562.0,7273.0,4562.0,5714.0
